# Latitude/Longitude Crossing Detector

Detectors for geographic latitude and longitude crossings.

These detectors identify when a spacecraft crosses a fixed latitude or longitude with respect to a central body (earth in this case).



In [3]:
import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()

from org.orekit.attitudes import NadirPointing
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.orekit.frames import FramesFactory
from org.orekit.time import TimeScalesFactory, AbsoluteDate
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from org.orekit.propagation import SpacecraftState
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.models.earth.tessellation import EllipsoidTessellator
from org.orekit.propagation.events import EventDetector, EventsLogger, LatitudeCrossingDetector, LongitudeCrossingDetector, BooleanDetector
from org.orekit.propagation.events.handlers import ContinueOnEvent, StopOnEvent

from math import radians, degrees, pi, sqrt, atan
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

First call the frames and time scales to be used in the code and create Earth body.

In [5]:
UTC = TimeScalesFactory.getUTC()                               # Define UTC time scale.
ECI = FramesFactory.getEME2000()                               # Define ECI reference frame.
ECEF = FramesFactory.getITRF(IERSConventions.IERS_2010, True)  # Define ECEF reference frame.
TEME = FramesFactory.getTEME()                                 # Define TEME reference frame. 
ITRF = ECEF

R_earth  = Constants.WGS84_EARTH_EQUATORIAL_RADIUS             # Radius of earth
Mu_earth = Constants.WGS84_EARTH_MU                            # Gravitational parameter of earth
f_earth  = Constants.WGS84_EARTH_FLATTENING                    # Earth flattening value

earth = OneAxisEllipsoid(R_earth, f_earth, ITRF)               # Create earth here.

Here, we first take TLE as an input, and from the TLEPropagator we recieve its PV to get Kepler Elements.

In [7]:
# ISS 25544
tle_line1 = "1 25544U 98067A   24258.18736906  .00032830  00000-0  59922-3 0  9997"
tle_line2 = "2 25544  51.6361 234.5150 0007532 350.4194   9.6649 15.49087105472352"

mytle = TLE(tle_line1, tle_line2)

initialDate = mytle.getDate()                    # This is the TLE epoch date read from first line.
finalDate = initialDate.shiftedBy(3600.0 * 10)   # Shift initial date by x hours.

print('Propagation initial date: ', initialDate)
print('Propagation final date: ', finalDate)

satellite_mass = 466615.0                        # Satellite mass [kg]
attitudeProvider = NadirPointing(TEME, earth)    # Also provide attitude of the satellite

SGP4 = TLEPropagator.selectExtrapolator(mytle, attitudeProvider, satellite_mass) 

Propagation initial date:  2024-09-14T04:29:48.686784Z
Propagation final date:  2024-09-14T14:29:48.686784Z


Prompts the user to input entry and exit latitudes and longitudes. It then converts the input values from degrees to radians for use in calculations involving spherical coordinates.

In [9]:
latitude_entry = input('Entry Latitude:  ')
latitude_entry = radians(float(latitude_entry))
latitude_exit = input('Exit Latitude:  ')
latitude_exit = radians(float(latitude_exit))

longitude_entry = input('Entry Longitude:  ')
longitude_entry = radians(float(longitude_entry))
longitude_exit = input('Exit Longitude:  ')
longitude_exit = radians(float(longitude_exit))

Entry Latitude:   0
Exit Latitude:   20
Entry Longitude:   0
Exit Longitude:   20


In [10]:
# Define a LatitudeCrossingDetector for "entrance" and add it to the propagator
lat_entry_Detector = LatitudeCrossingDetector(earth, latitude_entry).withHandler(ContinueOnEvent())
lat_entry_logger = EventsLogger()

# Define a LatitudeCrossingDetector for "exit" and add it to the propagator
lat_exit_Detector = LatitudeCrossingDetector(earth, latitude_exit).withHandler(ContinueOnEvent())
lat_exit_logger = EventsLogger()

# Define a LongitudeCrossingDetector for "entrance" and add it to the propagator
lon_entry_Detector = LongitudeCrossingDetector(earth, longitude_entry).withHandler(ContinueOnEvent())
lon_entry_logger = EventsLogger()

# Define a LongitudeCrossingDetector for "exit" and add it to the propagator
lon_exit_Detector = LongitudeCrossingDetector(earth, longitude_exit).withHandler(ContinueOnEvent())
lon_exit_logger = EventsLogger()

# Add detectors to the SGP4 propagator
SGP4.addEventDetector(lat_entry_logger.monitorDetector(lat_entry_Detector))
SGP4.addEventDetector(lat_exit_logger.monitorDetector(lat_exit_Detector))
SGP4.addEventDetector(lon_entry_logger.monitorDetector(lon_entry_Detector))
SGP4.addEventDetector(lon_exit_logger.monitorDetector(lon_exit_Detector))

If TLE is used, then we require a proper switch to Kepler Elements. This is done with Algorithm 4.2 from *Orbital Mechanics for Engineering Students, 3rd ed.* by Howard Curtis.

In [12]:
SGP4 = PVCoordinatesProvider.cast_(SGP4)

Mu = 398600                                                # Standart grav. parameter (km^3/s^2)
TLE_pv = SGP4.getPVCoordinates(initialDate, ECI)
pos = TLE_pv.getPosition()                                 # Get position vector
vel = TLE_pv.getVelocity()                                 # Get velocity vector
acc = TLE_pv.getAcceleration()                             # Get acceleration vector

pos = np.array([pos.getX(), pos.getY(), pos.getZ()])/1000  # Make position an array
vel = np.array([vel.getX(), vel.getY(), vel.getZ()])/1000  # Make velocity an array

pos_mag = sqrt(pos[0]**2 + pos[1]**2 + pos[2]**2)          # Find magnitude of position
vel_mag = sqrt(vel[0]**2 + vel[1]**2 + vel[2]**2)          # Find magnitude of velocity
velr= np.dot(vel, pos)/pos_mag                             # Find magnitude Vr

h = np.cross(pos, vel)                                     # Find specific angular momentum vector
h_mag = sqrt(h[0]**2 + h[1]**2 + h[2]**2)                  # Find specific angular momentum magnitude

i = np.arccos(h[2]/h_mag)                                  # Find the inclination (rad)

N = np.cross([0, 0, 1], h)                                 # Find "Node Line" vector
N_mag = sqrt(N[0]**2 + N[1]**2 + N[2]**2)                  # Find "Node Line" magnitude


if N[1]>=0:
    raan = np.arccos(N[0]/N_mag)                           # Find RAAN with this IF block
else:
    raan = radians(360 - degrees(np.arccos(N[0]/N_mag)))

e = (1/Mu)*((vel_mag**2-Mu/pos_mag)*pos-pos_mag*velr*vel)  # Find eccentricity vector
e_mag = sqrt(np.dot(e,e))                                  # Find eccentricity magnitude

if e[2]>=0:
    aop = np.arccos(np.dot(N,e)/(N_mag*e_mag))             # Find Argument of Perigee with this block
else:
    aop = radians(360 - degrees(np.arccos(np.dot(N,e)/(N_mag*e_mag))))
    
if velr>=0:
    ta = np.arccos(np.dot(e,pos)/(e_mag*pos_mag))          # Find True Anomaly with this IF block
else:
    ta = radians(360 - degrees(np.arccos(np.dot(e,pos)/(e_mag*pos_mag))))
    
a  = (h_mag**2/Mu)*(1/(1 - e_mag**2))*1000                 # Find Semimajor Axis in meters.

a = float(a)
e_mag = float(e_mag)
i = float(i)
aop = float(aop)
raan = float(raan)
ta = float(ta)

ra = a*(e_mag + 1)/1000                                    # Find the apoapsis value in km.  
ra = float(ra)

Now start propagating the TLE with SGP4. Here, propagation interval has to be sufficiently small because if it is large, detectors may miss certain events.

In [14]:
pos = []                                                       # position vector array to be filled.

###Start SGP4 propagation from initialDate up until finalDate
while (initialDate.compareTo(finalDate) <= 0.0):
    SGP4_pv = SGP4.getPVCoordinates(initialDate, ITRF)         # Get PV coordinates
    posSGP4 = SGP4_pv.getPosition()                            # But we only want position vector.
    pos.append((posSGP4.getX(),posSGP4.getY(),posSGP4.getZ())) # Get individual elements of position
    posSGP4 = pos
    initialDate = initialDate.shiftedBy(10.0)                  # Propagate with 10 sec intervals.

### Displaying Latitude/Longitude Crossing Events

Finally the logged events are retrieved and printed in a readable way.

In [17]:
lat_entry_events = lat_entry_logger.getLoggedEvents()

# Log latitude entries
lat_giris = [] 
for event in lat_entry_events: 
    lat_entry = str(event.getState().getDate())
    lat_giris.append(lat_entry)
    
df_giris = pd.DataFrame(lat_giris, columns=["Latitude Entries"])

lat_exit_events = lat_exit_logger.getLoggedEvents()

# Log latitude exits
lat_cikis = [] 
for event in lat_exit_events: 
    lat_exit = str(event.getState().getDate())
    lat_cikis.append(lat_exit)

df_cikis = pd.DataFrame(lat_cikis, columns=["Latitude Exits"])

# Ensure both dataframes have the same number of rows
min_length = min(len(df_giris), len(df_cikis))
df_giris = df_giris[:min_length]
df_cikis = df_cikis[:min_length]

# Concatenate the dataframes side by side
lat_df = pd.concat([df_giris, df_cikis], axis=1)

# Entry-Exit Order Adjustment
for i in range(len(lat_df)):
    if lat_df["Latitude Entries"][i] > lat_df["Latitude Exits"][i]:
        # Swap if entry is after exit
        lat_df.at[i, "Latitude Entries"], lat_df.at[i, "Latitude Exits"] = (
            lat_df.at[i, "Latitude Exits"],
            lat_df.at[i, "Latitude Entries"],
        )

# Apply styling for left alignment of the table
lat_df = lat_df.style.set_properties(**{'text-align': 'left'})
lat_df = lat_df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

# Display the resulting DataFrame
display(lat_df)


,Latitude Entries,Latitude Exits
0,2024-09-14T04:29:48.68764682977825Z,2024-09-14T04:36:26.07966280836906Z
1,2024-09-14T05:09:34.42924519521244Z,2024-09-14T05:16:12.96598308907777Z
2,2024-09-14T06:02:42.48118321600295Z,2024-09-14T06:09:19.87122399109186Z
3,2024-09-14T06:42:28.20561853388237Z,2024-09-14T06:49:06.74104102964237Z
4,2024-09-14T07:35:36.25931096678897Z,2024-09-14T07:42:13.64738568468478Z
5,2024-09-14T08:15:21.96658711653913Z,2024-09-14T08:22:00.50068421203234Z
6,2024-09-14T09:08:30.02202781416408Z,2024-09-14T09:15:07.40814562798948Z
7,2024-09-14T09:48:15.71214884049192Z,2024-09-14T09:54:54.24491053924176Z
8,2024-09-14T10:41:23.76933146745627Z,2024-09-14T10:48:01.15350154514813Z
9,2024-09-14T11:21:09.44230159528986Z,2024-09-14T11:27:47.97371790631957Z


In [18]:
# Log longitude entry events
lon_entry_events = lon_entry_logger.getLoggedEvents()
lon_giris = [] 
for event in lon_entry_events: 
    lon_entry = str(event.getState().getDate())
    lon_giris.append(lon_entry)
    
df_giris = pd.DataFrame(lon_giris, columns=["Longitude Entries"])

# Log longitude exit events
lon_exit_events = lon_exit_logger.getLoggedEvents()
lon_cikis = [] 
for event in lon_exit_events: 
    lon_exit = str(event.getState().getDate())
    lon_cikis.append(lon_exit)

df_cikis = pd.DataFrame(lon_cikis, columns=["Longitude Exits"])

# Ensure both dataframes have the same number of rows
min_length = min(len(df_giris), len(df_cikis))
df_giris = df_giris[:min_length]
df_cikis = df_cikis[:min_length]

# Concatenate the dataframes side by side
lon_df = pd.concat([df_giris, df_cikis], axis=1)

# Longitude Entry-Exit Order Adjustment
for i in range(len(lon_df)):
    if lon_df["Longitude Entries"][i] > lon_df["Longitude Exits"][i]:
        # Swap if entry is after exit
        lon_df.at[i, "Longitude Entries"], lon_df.at[i, "Longitude Exits"] = (
            lon_df.at[i, "Longitude Exits"],
            lon_df.at[i, "Longitude Entries"],
        )

# Apply styling for left alignment of the table
lon_df = lon_df.style.set_properties(**{'text-align': 'left'})
lon_df = lon_df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

# Display the resulting DataFrame
display(lon_df)


,Longitude Entries,Longitude Exits
0,2024-09-14T05:24:12.71088766888187Z,2024-09-14T05:30:32.7468632251297Z
1,2024-09-14T07:04:20.78486642172347Z,2024-09-14T07:08:33.00660350720993Z
2,2024-09-14T08:42:06.30554282074794Z,2024-09-14T08:45:31.70029489215825Z
3,2024-09-14T10:19:01.74572367050683Z,2024-09-14T10:22:34.06398196755389Z
4,2024-09-14T11:56:09.99867476008835Z,2024-09-14T12:00:47.89509437320942Z
5,2024-09-14T13:34:42.89565699888369Z,2024-09-14T13:41:53.1093184725007Z
